### Preprocessing


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:

# Load dataset
data=pd.read_csv("destinations_with_coordinates.csv")
df=pd.DataFrame(data)

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
scaler = MinMaxScaler()
data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']] = scaler.fit_transform(
    data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']]
)
# Add placeholder latitude and longitude columns if they don't exist
if 'latitude' not in data.columns:
    data['latitude'] = np.nan
if 'longitude' not in data.columns:
    data['longitude'] = np.nan

# Save the updated dataset
data.to_csv("destinations_with_coordinates.csv", index=False)

# Inspect data
print(data.tail())

     pID                pName  culture  adventure  wildlife  sightseeing  \
512  513       Chadani bridge      0.0        0.0       0.0          0.8   
513  514     Ghoda Ghodi lake      0.0        0.0       0.6          0.8   
514  515         Tikapur Park      0.0        0.0       0.0          0.7   
515  516  Godawari Ram Temple      0.8        0.0       0.0          0.8   
516  517       Karnali Bridge      0.0        0.0       0.0          0.8   

     history                            tags  province   latitude  longitude  
512      0.0                           river         7  28.923935  80.115051  
513      0.0  lake, wildlife, natural beauty         7        NaN        NaN  
514      0.7           park, history, forest         7  28.531705  81.120085  
515      0.0                   hindu, temple         7  27.601747  85.323626  
516      0.0                 river, scenery          7  28.640886  81.283829  


In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler



# Preprocess tags
data['tags'] = data['tags'].fillna('')  # Handle missing tags
data['tags_list'] = data['tags'].apply(lambda x: [tag.strip() for tag in x.split(',') if tag.strip()])

# Extract all unique tags and save to a file
all_tags = set(tag for tags in data['tags_list'] for tag in tags)
pd.DataFrame({'tag': list(all_tags)}).to_csv("tags.csv", index=False)

# Define tag-to-feature weights
tag_weights = {
    "historical": {"culture": 0.8, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 0.6, "history": 1.0},
    "wildlife": {"culture": 0.0, "adventure": 0.0, "wildlife": 1.0, "sightseeing": 0.3, "history": 0.0},
    "adventure": {"culture": 0.0, "adventure": 1.0, "wildlife": 0.5, "sightseeing": 0.5, "history": 0.0},
    "cultural": {"culture": 1.0, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 0.8, "history": 0.7},
    "scenic": {"culture": 0.2, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 1.0, "history": 0.2},
}

# Normalize numeric features
scaler = MinMaxScaler()
data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']] = scaler.fit_transform(
    data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']]
)




# Normalize numeric features
features = ['culture', 'adventure', 'wildlife', 'sightseeing', 'history']
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])


### model generation

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

# Extract features for similarity
features = ['culture', 'adventure', 'wildlife', 'sightseeing', 'history']

# Compute the destination similarity matrix
feature_matrix = data[features].values
similarity_matrix = cosine_similarity(feature_matrix)

### API integration

In [11]:
import requests

def get_weather(lat, lon):
    """
    Fetch weather information for a given latitude and longitude using OpenWeatherMap API.
    """
    api_key = "13e12c96cea3ae958db69de1f4bf41bd"  
    url = f"https://api.openweathermap.org/data/2.0/weather?lat={lat}&lon={lon}&appid={api_key}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            weather_data = response.json()
            return {
                "description": weather_data["weather"][0]["description"],
                "temperature": weather_data["main"]["temp"],
                "humidity": weather_data["main"]["humidity"],
            }
        else:
            print(f"API Error: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None


### recommendor

In [12]:

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


def recommend_destinations_with_weather(input_destination, user_preferences, selected_tags, top_n=5):
    """
    Recommend destinations based on cosine similarity, user preferences, and tags.
    Filter the recommendations by weather conditions.
    """
    # Define feature columns
    features = ["culture", "adventure", "wildlife", "sightseeing", "history"]
    feature_matrix = data[features].values

    # Find the index of the input destination
    if input_destination not in data['pName'].values:
        raise ValueError(f"Destination '{input_destination}' not found in dataset.")
    destination_idx = data[data['pName'] == input_destination].index[0]

    # Compute similarity scores
    similarity_scores = cosine_similarity(feature_matrix, feature_matrix[destination_idx].reshape(1, -1)).flatten()

    # Incorporate user preferences
    user_vector = np.array([user_preferences[feature] for feature in features])
    similarity_scores += cosine_similarity(feature_matrix, user_vector.reshape(1, -1)).flatten()

    # Boost scores based on tags
    tag_weights = {tag: 1 for tag in selected_tags}
    for idx, row in data.iterrows():
        row_tags = row['tags'].split(',')
        for tag in row_tags:
            if tag in tag_weights:
                similarity_scores[idx] += tag_weights[tag]

    # Sort destinations by adjusted scores
    data['similarity'] = similarity_scores
    sorted_data = data.sort_values(by='similarity', ascending=False)

    # Get top N recommendations
    recommendations = sorted_data.head(top_n)
    recommendations_with_weather = []

    # Add weather information
    for _, row in recommendations.iterrows():
        destination_info = {
            "pID": row['pID'],
            "pName": row['pName'],
            "tags": row['tags'],
            "similarity": row['similarity'],
        }
        lat, lon = row.get('latitude'), row.get('longitude')
        if pd.notnull(lat) and pd.notnull(lon):
            weather_data = get_weather(lat, lon)
            if weather_data:
                destination_info["weather"] = weather_data.get("description", "Weather info not available")
            else:
                destination_info["weather"] = "Weather info not available"
        else:
            destination_info["weather"] = "No weather data available"

        recommendations_with_weather.append(destination_info)

    return pd.DataFrame(recommendations_with_weather)

#user input
input_destination = "Jiri"
user_preferences = {"culture": 4, "adventure": 5, "wildlife": 3, "sightseeing": 5, "history": 2}
selected_tags = ["adventure", "culture"]

# Get recommendations
final_recommendations = recommend_destinations_with_weather(input_destination, user_preferences, selected_tags, top_n=5)
print("\nWeather-Filtered Recommendations:")
print(final_recommendations[['pID', 'pName','weather']])

API Error: 401, {"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}

Weather-Filtered Recommendations:
   pID                             pName                     weather
0  234              Ranipokhari, chitwan   No weather data available
1  232                     Bishazari Tal   No weather data available
2  448  Rafting and Kayaking in Karnali    No weather data available
3   74                       Tamur river  Weather info not available
4  246                   Larpak Village    No weather data available
